In [ ]:
!pip install transformers datasets accelerate rouge_score

In [ ]:
!pip install --upgrade datasets pyarrow

from datasets import load_dataset
import os

# Clear cache and set fresh environment
os.environ['HF_DATASETS_CACHE'] = '/tmp/hf_cache'
os.environ['HF_HOME'] = '/tmp/hf_home'

try:
    # Method 1: New official loading method
    dataset = load_dataset('cnn_dailymail', '3.0.0', trust_remote_code=True)
    print("Successfully loaded with Method 1")
except Exception as e:
    print(f"Method 1 failed: {e}")
    try:
        # Method 2: Alternative loading
        dataset = load_dataset('cnn_dailymail.py', trust_remote_code=True)
        print("Successfully loaded with Method 2")
    except Exception as e:
        print(f"Method 2 failed: {e}")
        try:
            # Method 3: Load from parquet files
            dataset = load_dataset('parquet', data_files={
                'train': 'https://huggingface.co/datasets/cnn_dailymail/resolve/main/train-00000-of-00001.parquet',
                'validation': 'https://huggingface.co/datasets/cnn_dailymail/resolve/main/validation-00000-of-00001.parquet',
                'test': 'https://huggingface.co/datasets/cnn_dailymail/resolve/main/test-00000-of-00001.parquet'
            })
            print("Successfully loaded with Method 3")
        except Exception as e:
            print(f"All methods failed: {e}")
            print("Please try in a fresh Colab runtime or virtual environment")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Successfully loaded with Method 1


In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")

max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    # Tokenize targets (summaries)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["highlights"],
            max_length=max_target_length,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the preprocessing to the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
# from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer

# model = T5ForConditionalGeneration.from_pretrained("t5-small")

# training_args = TrainingArguments(
#     output_dir="./results",
#     eval_strategy="steps",
#     eval_steps=1000,
#     logging_steps=500,
#     save_steps=1000,
#     save_total_limit=2,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=1,  # Use more epochs if needed
#     weight_decay=0.01,
#     fp16=True,
#     push_to_hub=False,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets["train"].select(range(20000)),  # You can change this for a quick run
#     eval_dataset=tokenized_datasets["validation"].select(range(1000)),
#     tokenizer=tokenizer,
# )
# trainer.train()
# #for epoch=1

<ipython-input-11-73ddde3351c6>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss
1000,1.076800,0.842544
2000,1.062700,0.837283


TrainOutput(global_step=2500, training_loss=1.1436334716796874, metrics={'train_runtime': 544.5328, 'train_samples_per_second': 36.729, 'train_steps_per_second': 4.591, 'total_flos': 2706836029440000.0, 'train_loss': 1.1436334716796874, 'epoch': 1.0})

In [ ]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer

model = T5ForConditionalGeneration.from_pretrained("t5-small")

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    eval_steps=1000,
    logging_steps=500,
    save_steps=1000,
    save_total_limit=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,  # Use more epochs if needed
    weight_decay=0.01,
    fp16=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].select(range(20000)),  # You can change this for a quick run
    eval_dataset=tokenized_datasets["validation"].select(range(1000)),
    tokenizer=tokenizer,
)
trainer.train()
#for epoch=5

<ipython-input-12-22eaa5f6c28d>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss
1000,1.075200,0.843430
2000,1.059500,0.839035
3000,1.038600,0.834846
4000,1.037600,0.833102
5000,1.026600,0.835025
6000,1.013900,0.834390
7000,1.025800,0.834672
8000,1.013400,0.833575
9000,1.014600,0.833409
10000,1.005300,0.833025


TrainOutput(global_step=12500, training_loss=1.0424363916015624, metrics={'train_runtime': 2776.5542, 'train_samples_per_second': 36.016, 'train_steps_per_second': 4.502, 'total_flos': 1.35341801472e+16, 'train_loss': 1.0424363916015624, 'epoch': 5.0})

In [ ]:
# Save model
trainer.save_model("./t5-small-finetuned-cnn-epoch5")

In [ ]:
# # Save model
# trainer.save_model("./t5-small-finetuned-cnn")

# # # Generate summary for a sample
# # text = dataset["test"][0]["article"]
# # inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)

# # output_ids = model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
# # print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

In [ ]:
import shutil

# Zip the model folder
shutil.make_archive("t5-small-finetuned-cnn-epoch5", 'zip', "t5-small-finetuned-cnn-epoch5")


'/content/t5-small-finetuned-cnn-epoch5.zip'

In [ ]:
# import shutil

# # Zip the model folder
# shutil.make_archive("t5-small-finetuned-cnn", 'zip', "t5-small-finetuned-cnn")


'/content/t5-small-finetuned-cnn.zip'

In [ ]:
# from google.colab import files
# files.download("t5-small-finetuned-cnn.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download("t5-small-finetuned-cnn-epoch5.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>